# Precompiling Numba modules

One issue with Numba is that it can be hard to install.  With `conda` it's incredibly easy, but not everybody uses `conda` and trying to explain to users/collaborators why they're doing everything wrong is hard.  

Tools like SWIG can compile C/C++ (or other) code at install time and make it available as a Python module if there's some serious numerical heavy-lifting required.  

But if you have ever tried to use SWIG together with NumPy... suffice to say it's a less than ideal arrangement (please don't hurt me, @dabeaz).

## Ahead-of-Time compilation

While Numba's main use is in JIT compiling, they do provide tools for doing AOT compilation.  This pre-compiled module does not rely on Numba, only on NumPy.  (If you are working with collaborators who don't have NumPy installed, I can't help you).

We need to import `numpy`, of course, and also `numba.pycc.CC`

In [ ]:
import numpy
from numba.pycc import CC

Name the module `ppe` (I am not creative)

In [ ]:
cc = CC('ppe')
cc.verbose = True

In [ ]:
@cc.export('pressure_poisson', 
           'f8[:,:](f8[:,:], f8[:,:], f8)')
def pressure_poisson(p, b, l2_target=1e-3):
    pn = p.copy()
    J, I = b.shape

    iter_diff = l2_target + 1

    n = 0
    while iter_diff > l2_target:
        pn = p.copy()
        for j in range(1, J - 1):
            for i in range(1, I - 1):
                p[j, i] = (.25 * (pn[j, i + 1] +
                                  pn[j, i - 1] +
                                  pn[j + 1, i] +
                                  pn[j - 1, i]) -
                                  b[j, i])

        for j in range(J):
            p[j, 0] = p[j, 1]
            p[j, -1] = p[j, -2]

        for i in range(I):
            p[0, i] = p[1, i]
            p[-1, i] = 0

        if n % 10 == 0:
            iter_diff = (numpy.sum((p - pn)**2)/numpy.sum(pn**2))**.5
        if n == 500:
            break

        n += 1

    return p


In [ ]:
cc.compile()

In [ ]:
%ls